# Fake Reviews Detection

### Importing required libraries
Here I am going to importing some of the required library, if extra library is required to install It will be install later on.

In [64]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

### Inserting fake and real dataset

In [65]:
df_fake = pd.read_csv("fake_reviews.csv")
df_true = pd.read_csv("true_reviews.csv")

In [66]:
df_fake.head(5)

,reviewID,reviewerID,restaurantID,date,rating,reviewUsefulCount,reviewContent,flagged,name,location,...,coolCount,funnyCount,complimentCount,tipCount,fanCount,restaurantRating,mnr,rl,rd,Maximum Content Similarity
0,6iHxiTdunSlX1Fg_nGIEeg,MXfwbHv5T-U_lHix5p8cmQ,pbEiXam9YJL3neCYHGwLUA,7/24/2012,5,0,i finally made alinea i heard much alinea way ...,Y,Betty The Star R.,"Chicago, IL",...,3,3,0,0,0,4.5,0.083333,129,0.125,0.134237
1,ZrDMIIfvGb35_MQyZvcNlA,lszTWuvB2yQWCUjcRX73Xw,pbEiXam9YJL3neCYHGwLUA,07-12-2012,5,0,went friend s birthday all i say wow fine dini...,Y,Cindy S.,"Chicago, IL",...,1,0,0,0,0,4.5,0.166667,20,0.125,0.026265
2,vjSYh7i54mbhNPzjKW-75A,jxzkUDFAreOk3VpYFedxZA,WBU0yq9J8qiYQfI_fh2P1Q,8/24/2011,3,0,didn t chicken like folks come here instead i ...,Y,Rich G.,"Chicago, IL",...,3,1,1,0,0,4.0,0.083333,45,0.250,0.041505
3,XKjj-eKn6ZMkGcQ6E3OSXQ,PlKxCQX4aBNx2lp0EyHl0w,8d_DiHWB-pjBVW3a7D8EWg,2/24/2011,5,0,lives hype highly recommended,Y,Frank F.,"Chicago, IL",...,0,0,0,0,0,4.0,0.166667,4,0.250,0.000000
4,7EAzA1FCwmuapu_fI8PyiA,0TqpLe6svu1YUb6zKl2vHg,IvdOI86_CSCRFpid2atnyA,04-06-2011,3,0,it bad server nice little slow although bus pe...,Y,May H.,"Chicago, IL",...,0,0,0,0,0,4.0,0.083333,53,0.250,0.057218


In [67]:
df_true.head(5)

,reviewID,reviewerID,restaurantID,date,rating,reviewUsefulCount,reviewContent,flagged,name,location,...,coolCount,funnyCount,complimentCount,tipCount,fanCount,restaurantRating,mnr,rl,rd,Maximum Content Similarity
0,GtwU21YOQn-wf4vWRUIx6w,bNYesZ944s6IJVowOnB0iA,pbEiXam9YJL3neCYHGwLUA,9/22/2012,5,0,unlike next we d eaten previous night dish com...,N,Scott E.,"Glengary, WV",...,5,5,2,0,1,4.5,0.083333,497,0.125,0.123653
1,0LpVTc3,TRKxLC3y-ZvP45e5iilMtw,pbEiXam9YJL3neCYHGwLUA,9/22/2012,5,0,probably one best meals i ve ever it s perform...,N,Jerry K.,"Palos Verdes Peninsula, CA",...,0,0,0,0,0,4.5,0.083333,41,0.125,0.000000
2,tljtLzf68Fkwf,0EMm8umAqXZzyhxNpL4M9g,pbEiXam9YJL3neCYHGwLUA,9/19/2012,3,2,service impeccable experience presentation coo...,N,Patricia M.,"Chicago, IL",...,0,1,0,0,0,4.5,0.083333,27,0.375,0.000000
3,iSN,DlwexC7z88ymAzu45skODw,pbEiXam9YJL3neCYHGwLUA,09-06-2012,3,8,the problem places like this given exhorbitant...,N,Terry N.,"San Jose, CA",...,8,6,6,11,0,4.5,0.083333,244,0.375,0.000000
4,Jmwrh7,kW2dk1CWihmh3g7k9N2G8A,pbEiXam9YJL3neCYHGwLUA,09-09-2012,5,1,i idea write review dining alinea brings whole...,N,Shradha Vegetarianista A.,"Chicago, IL",...,679,417,283,22,70,4.5,0.083333,97,0.125,0.760866


Inserting a column called "class" for fake and real news dataset to categories fake and true news. 

In [68]:
df_fake["class"] = 0
df_true["class"] = 1

Removing last 10 rows from both the dataset, for manual testing  

In [69]:
df_fake.shape, df_true.shape

((6206, 26), (20752, 26))

In [70]:
df_fake_manual_testing = df_fake.tail(10)
for i in range(6205,6195,-1):
    df_fake.drop([i], axis = 0, inplace = True)
df_true_manual_testing = df_true.tail(10)
for i in range(20751,20741,-1):
    df_true.drop([i], axis = 0, inplace = True)

In [71]:
df_fake.shape, df_true.shape

((6196, 26), (20742, 26))

Merging the manual testing dataframe in single dataset and save it in a csv file

In [72]:
df_fake_manual_testing["class"] = 0
df_true_manual_testing["class"] = 1

C:\Users\vrlgo\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\vrlgo\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [73]:
df_fake_manual_testing.head(10)

,reviewID,reviewerID,restaurantID,date,rating,reviewUsefulCount,reviewContent,flagged,name,location,...,funnyCount,complimentCount,tipCount,fanCount,restaurantRating,mnr,rl,rd,Maximum Content Similarity,class
6196,zaJVpQGhfVlq3CEjiEuxCA,JuLx79eaCI61ydYmpO3AfQ,hvHVAs2lhaoYo_rxOUuSTQ,11/22/2010,5,0,la villa best i always end going home tomorrow...,Y,John S.,"Park Ridge, IL",...,2,0,0,0,4.0,0.083333,39,0.25,0.0,0
6197,o0Qq8z_GnUC47dTppUHbiw,p8v0TjOnnvwYnthgtFz2eQ,hvHVAs2lhaoYo_rxOUuSTQ,10/27/2010,5,0,when i lived chicago i would eat time amazing ...,Y,Kevin S.,"Fox Lake, IL",...,0,0,0,0,4.0,0.083333,16,0.25,0.0,0
6198,9Rv5XZdgPjgswig8GtSEsQ,ohLoDGwzBkVB-bKNOU_kQg,hvHVAs2lhaoYo_rxOUuSTQ,10-12-2011,1,0,wow threw away food i delivered we ordered two...,Y,emily s.,"Chicago, IL",...,0,0,0,0,4.0,0.083333,60,0.75,0.0,0
6199,Df1YtkgxltGedZd8ZthC2Q,LaKxzYnszB4KXiEnwT4wjg,hvHVAs2lhaoYo_rxOUuSTQ,8/17/2011,1,0,la villa way overpriced i paid 17 90 plate spa...,Y,Food D.,"Chicago, IL",...,0,0,0,0,4.0,0.083333,57,0.75,0.0,0
6200,KhleekPLZ5zPCg-lexunAg,dt6P5WXxJmVAhTWu2L-Z8Q,hvHVAs2lhaoYo_rxOUuSTQ,08-09-2011,1,0,i watched episode check please read reviews ye...,Y,Frank Dirty Frank P.,"Chicago, IL",...,2,0,0,0,4.0,0.083333,296,0.75,0.0,0
6201,6XBHMUYXI7ryKT0qAoDplQ,zRToq49ZqJlFmP8Vucv16A,hvHVAs2lhaoYo_rxOUuSTQ,7/15/2011,4,0,i ve going la villa 6 years i think really goo...,Y,ELIZABETH S.,"Chicago, IL",...,0,0,0,0,4.0,0.083333,64,0.00,0.0,0
6202,#NAME?,NVNl-K5xLqkWdW4yIF1AlQ,hvHVAs2lhaoYo_rxOUuSTQ,07-08-2011,5,0,i agree gwendolyn place awesome we first time ...,Y,Marcin O.,"Streamwood, IL",...,0,1,0,0,4.0,0.083333,59,0.25,0.0,0
6203,ejFYB5GvvDRLFr1GEgZ0JQ,O0zxqwd8UgmDB5Q3PRmWjw,hvHVAs2lhaoYo_rxOUuSTQ,5/25/2011,4,0,think red leather booths christmas lights year...,Y,Andy B.,"Hinsdale, IL",...,1,1,0,0,4.0,0.083333,124,0.00,0.0,0
6204,0obKOWwCwXls-8FsaQkv9g,_jJeHQkRg4UDlabuoLVf3g,hvHVAs2lhaoYo_rxOUuSTQ,05-02-2011,1,0,read positives reviews yelp decided give place...,Y,Wendy W.,"Chicago, IL",...,0,0,0,0,4.0,0.083333,35,0.75,0.0,0
6205,4ZbGLoMzCgS-W9OgPjXeJw,z4ho3S5sEcq8v0PDbmG-7w,hvHVAs2lhaoYo_rxOUuSTQ,4/26/2011,5,0,order every weekend sometimes twice week the l...,Y,Angel G T.,"Chicago, IL",...,1,0,0,0,4.0,0.083333,23,0.25,0.0,0


In [74]:
df_true_manual_testing.head(10)

,reviewID,reviewerID,restaurantID,date,rating,reviewUsefulCount,reviewContent,flagged,name,location,...,funnyCount,complimentCount,tipCount,fanCount,restaurantRating,mnr,rl,rd,Maximum Content Similarity,class
20742,xo0XbpW2boGa,P1oK3UGg1UjzwjvKh0qNqQ,0Cc0SBLU0joTqsnbitQ7Ng,4/20/2009,5,8,yes is miracle i want chilaquiles i went pick ...,N,Diana D Rod! R.,"Chicago, IL",...,620,737,1,11,4.5,0.083333,65,0.125,0.113941,1
20743,xrDSjkmJou8q6,an45f8z9YPJc8GTvkcGMQw,0Cc0SBLU0joTqsnbitQ7Ng,08-04-2009,5,1,ok get ready totally biased review i give el m...,N,Nino M.,"Chicago, IL",...,18,8,0,0,4.5,0.083333,141,0.125,0.049775,1
20744,ZKSfHPsAsyA8vP4ZI,gN4WQ6Ud3wqdL0Pnov1hpw,KaFSO0j50RdPEcIwpxwM8A,10-03-2010,3,0,great vegetarian burrito beans nonexistent,N,Kimme Darlin' O.,"Chicago, IL",...,23,7,2,1,4.0,0.083333,5,0.250,0.000000,1
20745,tmnVeNrTc,S7F2pdsGo81vH7iV5ioe7g,KaFSO0j50RdPEcIwpxwM8A,6/20/2008,5,0,very good vegetarian burrito everything fresh ...,N,Kimme the princess O.,"Chicago, IL",...,4,0,0,1,4.0,0.083333,23,0.250,0.092321,1
20746,GF5N5PyuX8Q7H,scyLZbo_fDTi9-hpyOZTHA,s0M7_FoBGVAP-Wmo3_oq6g,05-10-2012,5,3,i wanting try latest edition west rogers park ...,N,Karen Brayden's Mommy S.,"Skokie, IL",...,6,3,14,2,4.5,0.083333,151,0.125,0.214565,1
20747,PZu8sDx2T2,tivh8lr6pzBDNfrJLYWh_g,v9qEDxi3t-P0CmGWAkkGvw,3/17/2010,4,0,it s taco bell higher prices taco bell what st...,N,Kristen F.,"Chicago, IL",...,17,24,0,3,4.0,0.083333,48,0.000,0.111133,1
20748,S-zbPPGoB,jKs4FQgkV0wSX8BG2_dgTg,RRflazDtBkqqpvEz2hbV2w,4/14/2011,5,0,yellow rose favorite mine i d go every day i c...,N,Yvonne F.,IL,...,0,1,0,1,4.0,0.083333,63,0.250,0.139705,1
20749,roKqXYooTy49OMAIJJjf,vX6aOMQ3HWCbwZVfCkCauw,zI0E_yruu58ea-xq9aHi-w,9/23/2007,3,0,not bad we ate odd wicker seats inviting we at...,N,Saverio T.,"Chicago, IL",...,10,7,1,1,4.0,0.083333,27,0.250,0.050463,1
20750,FefmFaWa,vX6aOMQ3HWCbwZVfCkCauw,6XVXM78gBuU3gpq2hTOgJA,11/18/2011,3,0,we surprised eagerly roped pop trivia game upo...,N,Saverio T.,"Chicago, IL",...,10,7,1,1,4.0,0.083333,42,0.250,0.050463,1
20751,x8knvE6V8MkwT90wCV0f,OZTkqoi8_luhrL-mMj7O8A,Lr4tZOsttQT-BgFtUkUTaQ,5/18/2012,4,0,a gorgeous shy young teen asked owner could si...,N,J. C. R.,"Chicago, IL",...,4,6,14,0,4.5,0.083333,20,0.125,0.000000,1


In [75]:
df_manual_testing = pd.concat([df_fake_manual_testing,df_true_manual_testing], axis = 0)
df_manual_testing.to_csv("manual_testing.csv")

Merging the main fake and true dataframe

In [76]:
df_marge = pd.concat([df_fake, df_true], axis =0 )
df_marge.head(10)

,reviewID,reviewerID,restaurantID,date,rating,reviewUsefulCount,reviewContent,flagged,name,location,...,funnyCount,complimentCount,tipCount,fanCount,restaurantRating,mnr,rl,rd,Maximum Content Similarity,class
0,6iHxiTdunSlX1Fg_nGIEeg,MXfwbHv5T-U_lHix5p8cmQ,pbEiXam9YJL3neCYHGwLUA,7/24/2012,5,0,i finally made alinea i heard much alinea way ...,Y,Betty The Star R.,"Chicago, IL",...,3,0,0,0,4.5,0.083333,129,0.125,0.134237,0
1,ZrDMIIfvGb35_MQyZvcNlA,lszTWuvB2yQWCUjcRX73Xw,pbEiXam9YJL3neCYHGwLUA,07-12-2012,5,0,went friend s birthday all i say wow fine dini...,Y,Cindy S.,"Chicago, IL",...,0,0,0,0,4.5,0.166667,20,0.125,0.026265,0
2,vjSYh7i54mbhNPzjKW-75A,jxzkUDFAreOk3VpYFedxZA,WBU0yq9J8qiYQfI_fh2P1Q,8/24/2011,3,0,didn t chicken like folks come here instead i ...,Y,Rich G.,"Chicago, IL",...,1,1,0,0,4.0,0.083333,45,0.250,0.041505,0
3,XKjj-eKn6ZMkGcQ6E3OSXQ,PlKxCQX4aBNx2lp0EyHl0w,8d_DiHWB-pjBVW3a7D8EWg,2/24/2011,5,0,lives hype highly recommended,Y,Frank F.,"Chicago, IL",...,0,0,0,0,4.0,0.166667,4,0.250,0.000000,0
4,7EAzA1FCwmuapu_fI8PyiA,0TqpLe6svu1YUb6zKl2vHg,IvdOI86_CSCRFpid2atnyA,04-06-2011,3,0,it bad server nice little slow although bus pe...,Y,May H.,"Chicago, IL",...,0,0,0,0,4.0,0.083333,53,0.250,0.057218,0
5,AzAoPQFysE4wTVWdurM0Gw,mN25MamB30-gtCC-GbvSwA,IvdOI86_CSCRFpid2atnyA,1/22/2011,5,0,was last night booo ya,Y,Matt Matty Boy W.,"Chicago, IL",...,2,0,0,0,4.0,0.083333,5,0.250,0.000000,0
6,H9YMpyJNjZQa1a0J5hgQeQ,bcfG-w1oEhRSRzIcQm_Bxw,IvdOI86_CSCRFpid2atnyA,02-08-2012,4,0,i opportunity try place friends family recent ...,Y,Jimmy L.,"Lombard, IL",...,0,4,0,1,4.0,0.083333,85,0.000,0.047917,0
7,bgZYPS687aLvVN1Bhih6zQ,EbLoh6p0Vj8ZFKq2ZHsO7A,IvdOI86_CSCRFpid2atnyA,06-02-2011,4,0,last night second visit davanti enoteca i yet ...,Y,Lisa H.,"Evanston, IL",...,1,0,0,0,4.0,0.083333,109,0.000,0.167853,0
8,I73kzk9Q9t7v3B1LnM2y2g,qlGgHX-ODtXhqacSjqAl6w,IvdOI86_CSCRFpid2atnyA,8/15/2012,2,0,if looking authentic italian forget place i mo...,Y,Ross E.,"Chicago, IL",...,1,0,0,0,4.0,0.083333,105,0.500,0.000000,0
9,1YXdmOYRcA5-OMBeEUh6gg,oQIAl5_bKkbnT89qD_UTwg,IvdOI86_CSCRFpid2atnyA,08-08-2012,4,0,nice atmosphere i would liked eat outside hot ...,Y,John B.,"Oak Park, IL",...,0,0,0,0,4.0,0.083333,39,0.000,0.000000,0


In [77]:
df_marge.columns

Index(['reviewID', 'reviewerID', 'restaurantID', 'date', 'rating',
       'reviewUsefulCount', 'reviewContent', 'flagged', 'name', 'location',
       'yelpJoinDate', 'friendCount', 'reviewCount', 'firstCount',
       'usefulCount', 'coolCount', 'funnyCount', 'complimentCount', 'tipCount',
       'fanCount', 'restaurantRating', 'mnr', 'rl', 'rd',
       'Maximum Content Similarity', 'class'],
      dtype='object')

#### columns other than reviewContent and class are not required for detecting the fake reviews, so I am going to drop the columns.

In [78]:
df = df_marge.drop(df_marge.columns.difference(['reviewContent','class']), axis = 1)

In [94]:
df = df.dropna(axis=0, how="any", thresh=None, subset=None, inplace=False)

In [ ]:
df.isnull().sum()

reviewContent    0
class            0
dtype: int64

#### Randomly shuffling the dataframe 

In [80]:
df = df.sample(frac = 1)

In [81]:
df.head()

,reviewContent,class
2346,going zed451 always good time if looking great...,0
9111,i ordered deep dish pepperoni sausage mushroom...,1
13058,i ve never great brunch entire life even diggi...,1
10286,our team event private upstairs section overse...,1
3887,my husbands day work i thoughht i would surpri...,1


In [96]:
df.reset_index(inplace = True)
df.drop(["index"], axis = 1, inplace = True)

In [97]:
df.columns

Index(['reviewContent', 'class'], dtype='object')

In [84]:
df.head()

,reviewContent,class
0,going zed451 always good time if looking great...,0
1,i ordered deep dish pepperoni sausage mushroom...,1
2,i ve never great brunch entire life even diggi...,1
3,our team event private upstairs section overse...,1
4,my husbands day work i thoughht i would surpri...,1


#### Creating a function to convert the text in lowercase, remove the extra space, special chr., ulr and links.

In [98]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [99]:
df["reviewContent"] = df["reviewContent"].apply(wordopt)

#### Defining dependent and independent variable as x and y

In [100]:
x = df["reviewContent"]
y = df["class"]

#### Splitting the dataset into training set and testing set. 

In [101]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

#### Convert text to vectors

In [102]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [103]:
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

### 1. Logistic Regression

In [104]:
from sklearn.linear_model import LogisticRegression

In [105]:
LR = LogisticRegression()
LR.fit(xv_train,y_train)

LogisticRegression()

In [106]:
pred_lr=LR.predict(xv_test)

In [107]:
LR.score(xv_test, y_test)

0.7821502821502822

In [108]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.62      0.16      0.26      1564
           1       0.79      0.97      0.87      5170

    accuracy                           0.78      6734
   macro avg       0.71      0.57      0.56      6734
weighted avg       0.75      0.78      0.73      6734



### 2. Decision Tree Classification

In [109]:
from sklearn.tree import DecisionTreeClassifier

In [110]:
DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [111]:
pred_dt = DT.predict(xv_test)

In [112]:
DT.score(xv_test, y_test)

0.6786456786456786

In [113]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       0.31      0.30      0.30      1564
           1       0.79      0.79      0.79      5170

    accuracy                           0.68      6734
   macro avg       0.55      0.55      0.55      6734
weighted avg       0.68      0.68      0.68      6734



### 3. Gradient Boosting Classifier

In [114]:
from sklearn.ensemble import GradientBoostingClassifier

In [115]:
GBC = GradientBoostingClassifier(random_state=0)
GBC.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [116]:
pred_gbc = GBC.predict(xv_test)

In [117]:
GBC.score(xv_test, y_test)

0.772052272052272

In [118]:
print(classification_report(y_test, pred_gbc))

              precision    recall  f1-score   support

           0       0.66      0.04      0.07      1564
           1       0.77      0.99      0.87      5170

    accuracy                           0.77      6734
   macro avg       0.72      0.52      0.47      6734
weighted avg       0.75      0.77      0.68      6734



### 4. Random Forest Classifier

In [119]:
from sklearn.ensemble import RandomForestClassifier

In [120]:
RFC = RandomForestClassifier(random_state=0)
RFC.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [121]:
pred_rfc = RFC.predict(xv_test)

In [122]:
RFC.score(xv_test, y_test)

0.7661122661122661

In [123]:
print(classification_report(y_test, pred_rfc))

              precision    recall  f1-score   support

           0       0.46      0.04      0.07      1564
           1       0.77      0.99      0.87      5170

    accuracy                           0.77      6734
   macro avg       0.61      0.51      0.47      6734
weighted avg       0.70      0.77      0.68      6734



# Model Testing With Manual Entry

### Reviews

In [130]:
def output_lable(n):
    if n == 0:
        return "Fake Review"
    elif n == 1:
        return "Not A Fake Review"
    
def manual_testing(news):
    testing_news = {"reviewContent":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["reviewContent"] = new_def_test["reviewContent"].apply(wordopt) 
    new_x_test = new_def_test["reviewContent"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GBC = GBC.predict(new_xv_test)
    pred_RFC = RFC.predict(new_xv_test)

    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGBC Prediction: {} \nRFC Prediction: {}".format(output_lable(pred_LR[0]), 
                                                                                                              output_lable(pred_DT[0]), 
                                                                                                              output_lable(pred_GBC[0]), 
                                                                                                              output_lable(pred_RFC[0])))

In [131]:
reviews = str(input())
manual_testing(reviews)



LR Prediction: Not A Fake Review 
DT Prediction: Fake Review 
GBC Prediction: Not A Fake Review 
RFC Prediction: Fake Review
